<a href="https://colab.research.google.com/github/devinbook/Tagalog-profanity-detection/blob/main/ScrapingReddit_test_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw

In [ ]:
import praw
import re
import time
import pandas as pd

In [ ]:
reddit = praw.Reddit(client_id='uLfq7N-Kp-n_noaXUDVuTg',
                     client_secret='IUhw5TKGpWeRI2yweNOEwuzeIVVi8A',
                     user_agent='Profanity_tagalog')

In [ ]:
tagalog_subreddits = ['Philippines', 'Tagalog', 'pinoy', 'offmychestph']

In [ ]:
def extract_profanity_snippet(text, keyword, words_before=10, words_after=10):
    words = text.split()
    try:
        keyword_index = next(i for i, word in enumerate(words) if keyword.lower() in word.lower())
    except StopIteration:
        return ''

    # Range ng snippet
    start = max(0, keyword_index - words_before)
    end = min(len(words), keyword_index + words_after + 1)

    return ' '.join(words[start:end])

def scrape_profanity(keyword, subreddits, max_posts=3000, words_before=10, words_after=10):
    posts_scraped = 0
    post_data = []

    for subreddit_name in subreddits:
        print(f"Searching in subreddit: {subreddit_name} for keyword: {keyword}")

        # Iterator para macheck bawat reddit
        submission_iterator = reddit.subreddit(subreddit_name).search(keyword, sort='new', limit=100)

        while posts_scraped < max_posts:
            try:
                for post in submission_iterator:
                    combined_text = post.title + " " + post.selftext
                    profanity_snippet = extract_profanity_snippet(combined_text, keyword, words_before, words_after)

                    if profanity_snippet:
                        post_data.append({
                            "Text": profanity_snippet,
                            "Label": 1,
                            "Profanity Word": keyword,
                            "Source": subreddit_name,
                            "Timestamp": post.created_utc,
                            "Upvotes": post.score,
                            "URL": post.url,
                            "Subreddit": post.subreddit.display_name
                        })
                        posts_scraped += 1

                    if posts_scraped >= max_posts:
                        break

                if posts_scraped >= max_posts:
                    break  # Exit pag max na yung post

                if posts_scraped < max_posts:
                    print(f"Posts scraped for {keyword}: {posts_scraped}/{max_posts}. Pausing for 1.1 seconds to avoid rate limits.")
                    time.sleep(1.1)
                    submission_iterator = reddit.subreddit(subreddit_name).search(keyword, sort='new', limit=100)

            except praw.exceptions.APIException as e:
                print(f"API limit reached, sleeping for {e.sleep_time} seconds.")
                time.sleep(e.sleep_time)
            except Exception as e:
                print(f"An error occurred in subreddit {subreddit_name}: {e}")
                break

    return post_data

profanity_words = [
    "gago", "tanga", "bobo", "puta", "ulol", "bwisit",
    "tangina", "pakyu", "hayop", "tarantado", "putangina"
]
all_scraped_data = []

for word in profanity_words:
    print(f"Scraping data for profanity word: {word}")
    scraped_data = scrape_profanity(word, tagalog_subreddits, max_posts=2000, words_before=10, words_after=10)
    all_scraped_data.extend(scraped_data)
    print(f"Total posts scraped for {word}: {len(scraped_data)}")

In [ ]:
df = pd.DataFrame(all_scraped_data)

In [ ]:
df.head()

,Text,Label,Profanity Word,Source,Timestamp,Upvotes,URL,Subreddit
0,"kids. He, more often than not, hurtfully calls...",1,tanga,Filipino,1.720022e+09,1,https://www.reddit.com/r/Filipino/comments/1du...,Filipino
1,"kids. He, more often than not, hurtfully calls...",1,tanga,Filipino,1.720022e+09,1,https://www.reddit.com/r/Filipino/comments/1du...,Filipino
2,"kids. He, more often than not, hurtfully calls...",1,tanga,Filipino,1.720022e+09,1,https://www.reddit.com/r/Filipino/comments/1du...,Filipino
3,"kids. He, more often than not, hurtfully calls...",1,tanga,Filipino,1.720022e+09,1,https://www.reddit.com/r/Filipino/comments/1du...,Filipino
4,"kids. He, more often than not, hurtfully calls...",1,tanga,Filipino,1.720022e+09,1,https://www.reddit.com/r/Filipino/comments/1du...,Filipino


In [ ]:
df.shape

(2000, 8)

In [ ]:
df_cleaned = df.drop_duplicates(subset=['Text'], keep='first')

In [ ]:
df_cleaned.shape

(1, 8)

In [ ]:
df_cleaned.to_csv('11.csv', index=False)
from google.colab import files
files.download('11.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>